In [1]:
# Load necessary packages
import jax
import jax.numpy as jnp
import numpy as np
import optax
import matplotlib.pyplot as plt

# User-defined
from ANN_1DOF import Damped_MLP, Damped_LNN
from helpers import train_test_data, compare_sols, format_to_LNN

from jax import config
config.update("jax_enable_x64", True)

#### Generate Training Data

In [2]:
# Generate continuation data
old_train_dataset, old_test_dataset, info = train_test_data(
    save_file='data/data.pkl', 
    split_size=0.1, 
    file_name='contparameters.json', 
    min_force_amp=0.1, 
    max_force_amp=1.0, 
    step=0.1, 
    phase_ratio=0.5, 
    damping=0.05
)



Iter Cont     Iter Corr     Residual      Freq          Energy        Step          Beta          
0             0             1.2826e-01    0.0955        5.0861e-02                                
0             1             4.2430e-03    0.0955        1.9854e-02                                
0             2             9.4199e-07    0.0955        1.9376e-02                                
1             0             8.2709e-07    0.0955        1.9391e-02    -5.000e-03                  
1             1             1.9545e-13    0.0955        1.9392e-02    -5.000e-03    0.0022        
--------------------------------------------------------------------------------------------------
2             0             5.4762e-07    0.0956        1.9423e-02    -1.000e-02                  
2             1             1.2469e-13    0.0956        1.9423e-02    -1.000e-02    0.0039        
--------------------------------------------------------------------------------------------------
3       

/home/aax23/.conda/envs/.duffing/lib/python3.12/site-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
/home/aax23/PyCont_release/core/solver/_psacont.py:104: LinAlgWarning: Ill-conditioned matrix (rcond=8.19791e-276): result may not be accurate.
  dxt = spl.solve(Jcr, -Z, check_finite=True)
/home/aax23/PyCont_release/core/solver/_psacont.py:104: LinAlgWarning: Ill-conditioned matrix (rcond=1.95906e-20): result may not be accurate.
  dxt = spl.solve(Jcr, -Z, check_finite=True)


91            2             1.0684e+03    -0.0018       2.0817e+11    -1.000e-01                  
91            3             1.4322e+03    -0.0018       6.8092e+11    -1.000e-01                  
91            4             3.8776e+03    -0.0018       3.5271e+13    -1.000e-01                  
91            0             7.4145e-03    0.3216        7.2016e+00    -5.000e-02                  
91            1             1.0837e-02    0.3344        8.4393e+00    -5.000e-02                  
91            2             2.5280e-01    0.2752        4.2407e+00    -5.000e-02                  
91            3             7.8531e-02    0.2969        5.1836e+00    -5.000e-02                  
91            4             3.4761e-02    0.3151        6.7238e+00    -5.000e-02                  
91            0             3.8241e-03    0.3214        7.1801e+00    -2.500e-02                  
91            1             3.4493e-03    0.3288        7.8711e+00    -2.500e-02                  
91        

#### Format Dataset for LNN Code

In [4]:
old_train_dataset['x'].shape, old_train_dataset['dx'].shape, old_train_dataset['ddx'].shape, old_train_dataset['t'].shape, old_train_dataset['f'].shape

((270, 1873), (270, 1873), (270, 1873), (270, 1873), (270, 1873))

In [5]:
train_dataset, test_dataset, info = format_to_LNN(old_train_dataset, old_test_dataset, info)

#### LNN Training

In [ ]:
settings = {
    'name': 'ANN_LNN_Damped_Test',
    'lag_units': 64,
    'damp_units': 16,
    'layers': 3,
    'input_shape': 2,
    'train_batch_size': 128,
    'test_batch_size': 64,
    'shuffle': True,
    'seed': 0
    }

phy_sys = {
    'M': info['M'],
    'K': info['K'],
    'C': info['C'],
    'NL': info['NL'],
}

lr = 1e-03
optimizer = optax.adam(lr)
a = Damped_LNN(Damped_MLP, optimizer, settings, info, phy_sys)
a.gather()

In [ ]:
# # Load previous results, if any
# prev_results = a.load_model('results/ANN_Damped_100epochs/model.pkl')
# results = prev_results

In [ ]:
# Start training LNN
results = a.train(train_dataset, test_dataset, results=None, epochs=100, show_every=10)

In [ ]:
# Save results
a.save_model(results, 'results/ANN_Damped_100epochs')

#### Examine Data Scope

In [ ]:
import pickle
with open('./data/data.pkl', 'rb') as f:
   data = pickle.load(f)

In [ ]:
# Store ML data
x = np.array([])
dx = np.array([])
ddx = np.array([])
t = np.array([])
f = np.array([])
period = np.array([])

# Loop over data
for k, v in data.items():
    pose = data[k]["pose"]
    vel = data[k]["vel"]
    acc = data[k]["acc"]
    time = data[k]["time"]
    force = data[k]["force"]
    T = data[k]["T"]
        
    x = np.append(x, pose)
    dx = np.append(dx, vel)
    ddx = np.append(ddx, acc)
    t = np.append(t, time)
    f = np.append(f, force)
    period = np.append(period, T)

In [ ]:
# Plot figures
fig, ax = plt.subplots(3, 1, figsize=(10, 10))

# Plot Displacement, Velocity and Acceleration
ax[0].boxplot(x, vert=0)
ax[0].set_xlabel(r'$x$', fontsize=12)

ax[1].boxplot(dx, vert=0)
ax[1].set_xlabel(r'$\dot{x}$', fontsize=12)

ax[2].boxplot(ddx, vert=0)
ax[2].set_xlabel(r'$\ddot{x}$', fontsize=12)

### Examine results

In [ ]:
# Plotting the corresponding lagrangian and damping function for each output in the test dataset
pred_acc_damped, pred_energy_damped = a._predict(results)
q, q_d = jnp.split(test_dataset[0], 2, axis=-1)
n = 301
Lnn, Dnn = pred_energy_damped(q, q_d)

In [ ]:
# Comparing accelerations
F = test_dataset[1].reshape(q.shape[0], -1)
q_dd = pred_acc_damped(test_dataset[0], F)[:, -1]

In [ ]:
lim1, lim2 = info['qmax'], info['qdmax']

qa, qda = jnp.linspace(-lim1, lim1, 100), jnp.linspace(-lim2, lim2, 100)
qaa, qdaa = jnp.meshgrid(qa, qda)

# Get all energy functions here
L, D = jax.vmap(pred_energy_damped)(qaa.reshape(-1,1,1), qdaa.reshape(-1,1,1))
Lagrange_analy = 0.5*phy_sys['M']*qdaa**2 - 0.5*phy_sys['K']*qaa**2 - 0.25*phy_sys['NL']*qaa**4
Dissipation_analy = 0.5*phy_sys['C']*qdaa**2

In [ ]:
fig = plt.figure(figsize=(15,12), tight_layout=True)

ax = fig.add_subplot(221, projection='3d')
m = ax.plot_surface(qaa, qdaa, L.reshape(qaa.shape), alpha=0.5, cmap='plasma')
ax.contour3D(qaa, qdaa, L.reshape(qaa.shape), cmap='binary')
ax.set_xlabel('q')
ax.set_ylabel(r'$\dot{q}$')
ax.set_zlabel(r'$\mathcal{L}_{NN}$', fontsize=16, labelpad=3)
ax.set_title('Lagrangian prediction - overall phase space')
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(222, projection = '3d')
m = ax.plot_surface(qaa, qdaa, Lagrange_analy.reshape(qaa.shape), alpha=0.5, cmap='plasma')
ax.contour3D(qaa, qdaa, Lagrange_analy.reshape(qaa.shape), cmap='binary')
ax.set_title('Analytical Lagrangian')
ax.set_xlabel(r'$q \ (m)$', fontsize=12)
ax.set_ylabel(r'$\dot{q} \ (m \ s^{-1})$ ', fontsize=12)
ax.set_zlabel(r'L (J)', fontsize=14, labelpad=2)
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(223, projection='3d')
m = ax.plot_surface(qaa, qdaa, D.reshape(qaa.shape), alpha=0.7, cmap='magma')
ax.contour3D(qaa, qdaa, D.reshape(qaa.shape), cmap='binary')
ax.set_xlabel('q')
ax.set_ylabel(r'$\dot{q}$')
ax.set_zlabel(r'$\mathcal{D}_{NN}$', fontsize=16, labelpad=3)
ax.set_title('Dissipation prediction - overall phase space')
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

ax = fig.add_subplot(224, projection = '3d')
m = ax.plot_surface(qaa, qdaa, Dissipation_analy.reshape(qaa.shape), alpha=0.7, cmap='magma')
ax.contour3D(qaa, qdaa, Dissipation_analy.reshape(qaa.shape), cmap='binary')
ax.set_title('Analytical Dissipation')
ax.set_xlabel(r'$q \ (m)$', fontsize=12)
ax.set_ylabel(r'$\dot{q} \ (m \ s^{-1})$ ', fontsize=12)
ax.set_zlabel(r'D (J)', fontsize=14, labelpad=2)
fig.colorbar(m, ax=ax,shrink=0.5, pad=0.075)

#### Compare Analytical vs LNN FRCs

In [ ]:
compare_sols(anal_file='data_old/FRF6', lnn_file='data_LNN_old/FRF6')

In [ ]:
for i in range(1, 11, 1):
    compare_sols(anal_file=f'data_old/FRF{i}', lnn_file=f'data_LNN_old/FRF{i}')